In [1]:

from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer

In [2]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram
import meshcat

# Build plant

In [3]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()

# Setup meshcat visualization

In [4]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph)
diagram = visualizer.diagram

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


# Set up IRIS options

In [5]:
iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 5
iris_options.termination_threshold = -1
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False
# uncomment to test if non-default is working
# currently throwing std::bad_alloc if we try to set q_star
# iris_options.set_q_star(np.zeros(plant.num_positions()))


#seed points specified in q space
seed_points = np.array([
                        [0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]      # out of shelf 1
                        ])


In [6]:
regions = []
for i, s in enumerate(seed_points):
    context = diagram.CreateDefaultContext()
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
    r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context), iris_options)
    print(f"{r.A()}t <= {r.b()}")
    regions.append(r)
    print(f'Completed region: {i+1}/{len(seed_points)}')

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [-1. -0. -0.]
 [-0. -1. -0.]
 [-0. -0. -1.]]t <= [1.73205081 1.73205081 1.73205081 1.73205081 1.73205081 1.73205081]
Completed region: 1/6
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [-1. -0. -0.]
 [-0. -1. -0.]
 [-0. -0. -1.]]t <= [1.73205081 1.73205081 1.73205081 1.73205081 1.73205081 1.73205081]
Completed region: 2/6
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [-1. -0. -0.]
 [-0. -1. -0.]
 [-0. -0. -1.]]t <= [1.73205081 1.73205081 1.73205081 1.73205081 1.73205081 1.73205081]
Completed region: 3/6
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [-1. -0. -0.]
 [-0. -1. -0.]
 [-0. -0. -1.]]t <= [1.73205081 1.73205081 1.73205081 1.73205081 1.73205081 1.73205081]
Completed region: 4/6
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [-1. -0. -0.]
 [-0. -1. -0.]
 [-0. -0. -1.]]t <= [1.73205081 1.73205081 1.73205081 1.73205081 1.73205081 1.73205081]
Completed region: 5/6
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [-1. -0. -0.]
 [-0. -1. -0.]


In [7]:
# plot regions and collision constraint
if do_viz:
    visualizer.plot_regions(regions)
    visualizer.plot_seedpoints(seed_points)
    visualizer.visualize_collision_constraint(N = 50)